In [12]:
import os
from music21 import converter, instrument, note, chord

notes = []
midi_path = r"C:\Users\ML_DL\NeuralNets\RNN_Trials\LSTM_np\midi_dataset\beeth"

for file in os.listdir(midi_path):
    if file.endswith('.mid') or file.endswith('.midi'):
        full_path = os.path.join(midi_path, file)
        try:
            midi = converter.parse(full_path)
            notes_to_parse = None
            parts = instrument.partitionByInstrument(midi)
            if parts: 
                notes_to_parse = parts.parts[0].recurse()
            else: 
                notes_to_parse = midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")

C:\Users\dhruva rao\AppData\Roaming\Python\Python310\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2001 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
C:\Users\dhruva rao\AppData\Roaming\Python\Python310\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2008 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\dhruva rao\AppData\Roaming\Python\Python310\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2009 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\dhruva rao\AppData\Roaming\Python\Python310\site-package

In [ ]:
%pip install keras tensorflow
import tensorflow
import np as np 
from keras.utils import to_categorical

sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
n_vocab = len(set(notes))
# normalize input
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.layers import  LSTM, Dropout



model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
 ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


C:\Users\dhruva rao\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint  

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)


Epoch 1/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 715s 9s/step - loss: 3.8278
Epoch 2/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 1325s 17s/step - loss: 3.7755
Epoch 3/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 848s 11s/step - loss: 3.7242
Epoch 4/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 1040s 14s/step - loss: 3.6564
Epoch 5/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 3357s 44s/step - loss: 3.5979
Epoch 6/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 795s 10s/step - loss: 3.5585
Epoch 7/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 670s 9s/step - loss: 3.4674
Epoch 8/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 786s 10s/step - loss: 3.3740
Epoch 9/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 716s 9s/step - loss: 3.3714
Epoch 10/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 579s 8s/step - loss: 3.2795


In [31]:
from tensorflow.keras.models import load_model
model = load_model(r"C:\Users\ML_DL\NeuralNets\RNN_Trials\LSTM_np\LSTMusica\weights-improvement-10-3.2928-bigger.keras")

In [37]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern, index)

    pattern = pattern[1:len(pattern)]


In [38]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5


In [43]:
from music21 import stream, note, chord, instrument

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_2.mid')


'test_2.mid'